In [3]:
#Chargement des données
# Data
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os 
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import matplotlib.pyplot as plt
import math
# Graphics
import seaborn as sns ; sns.set()


from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import FunctionTransformer
# from sklearn.metrics import root_mean_squared_error

In [15]:
load_dotenv('BDD_URL.env')
BDD_URL = os.environ['BDD_URL']
engine = create_engine(BDD_URL)

SQL_filtre= """
SET search_path to principal;
SELECT *
from "filmview"
where 'Comedy' = ANY(string_to_array("genres", ','))
limit 10000;
"""
SQL= """
SET search_path to principal;
SELECT *
from "filmview"
where "runtimeMinutes" Is NOT null and "titleType" = 'movie'
limit 10000;
"""
df = pd.read_sql(SQL, engine)
engine.dispose()
df[df['isAdult'] == 1]

,tconst,primaryTitle,titleType,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,Cate&names
71,tt0095787,Only the Strong Survive,movie,1,1988,None,85,Adult,6.3,142,"[actor_Buddy_Love, actress_Bella_Donna, actres..."
232,tt0096038,Saturday Night Special,movie,1,1992,None,84,Adult,6.4,21,"[actress_Sharon_Kane, actress_Jenny_James, dir..."
555,tt0096749,Abatjour 2,movie,1,1990,None,100,"Crime,Mystery,Romance",5.4,31,"[production_designer_Max_Serafian, actress_Val..."
564,tt0096763,Adultery,movie,1,1990,None,81,"Adult,Drama,Romance",5.9,29,"[actress_Ona_Zee, director_Anthony_Spinelli, a..."
635,tt0096893,Bare Essence,movie,1,1989,None,92,Adult,6.0,17,"[actor_Dave_Bolden, actress_Champagne, directo..."
727,tt0097039,The Chameleon,movie,1,1989,None,85,"Adult,Drama,Fantasy",6.5,289,"[writer_Nick_Hunter, actress_Tori_Welles, cine..."
759,tt0097096,Coming of Age,movie,1,1989,None,85,Adult,5.6,128,"[actress_Tori_Welles, writer_Patti_Rhodes-Linc..."
812,tt0097175,Debbie Class of '89,movie,1,1989,None,77,Adult,6.6,113,"[actor_Jon_Dough, actress_Kim_Angeli, actor_Ro..."
1583,tt0098451,Taylor Made,movie,1,1989,None,82,Adult,6.4,108,"[actor_Peter_North, actor_Robert_Bullock, actr..."
1612,tt0098497,Torrid House,movie,1,1989,None,76,Adult,6.2,26,"[actress_Tamara_Lee, actor_Robert_Bullock, act..."


In [5]:
def listTostr (df):
    return df.apply(lambda x: ' '.join(map(str, x)))

In [10]:
# df["averageRating"].fillna(df["averageRating"].mean(), inplace=True)
# df["startYear"].fillna(df["startYear"].mean(), inplace=True)
# df["runtimeMinutes"].fillna(df["runtimeMinutes"].mean(), inplace=True)
# df["isAdult"] = df["isAdult"].apply(lambda x: 1 if x == "True" else 0)
df['isAdult'] = df['isAdult'].astype(bool)
df['genres'].fillna('missing', inplace=True)
df['Cate&names'].fillna('missing', inplace=True)
df['Cate&names']=df[['Cate&names']].apply(listTostr)
df[df["isAdult"] == True]


C:\Users\cleme\AppData\Local\Temp\ipykernel_8792\3851327344.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['genres'].fillna('missing', inplace=True)
C:\Users\cleme\AppData\Local\Temp\ipykernel_8792\3851327344.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, 

,tconst,primaryTitle,titleType,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,Cate&names


In [16]:
# Séparation des caractéristiques et de la cible
X = df.drop(columns=["averageRating"])
y = df["averageRating"]

# Séparation des données d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [17]:
# Définition des colonnes numériques, textuelles et de description
numeric_features = ['startYear', 'runtimeMinutes']
boolean_features = 'isAdult'
text_features = ['titleType']#['titleType', 'directors', 'writers']
genre = 'genres'
description_feature = 'Cate&names'

In [18]:
# Création des transformers pour les colonnes numériques, booléennes, textuelles et de description
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

boolean_transformer = FunctionTransformer(lambda x: x.astype(bool).values.reshape(-1, 1)) 

text_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

Vect_transformer = Pipeline([
    # ('imputer', SimpleImputer(strategy='constant', fill_value='missing')), 
    ('vect', CountVectorizer())
])

# Création d'un ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('bool', boolean_transformer, boolean_features),
        ('text', text_transformer, text_features),
        ('genre', Vect_transformer, genre),
        ('description', Vect_transformer, description_feature)
    ])
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['startYear', 'runtimeMinutes']),
                                ('bool',
                                 FunctionTransformer(func=<function <lambda> at 0x000001B5ABA7B420>),
                                 'isAdult'),
                                ('text',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['titleType']),
                                ('genre',
                                 Pipeline(steps=[('vect', CountVectorizer())]),
                                 'genres'),
                                ('description',
                                 Pipeline(steps=[('vect', CountVectorizer())]),
                                 'Cate&names')])

In [19]:
# Création des pipelines pour chaque modèle
pipelines = {
    'Linear Regression': Pipeline([('preprocessor', preprocessor), ('regressor', LinearRegression())]),
    'Ridge Regression': Pipeline([('preprocessor', preprocessor), ('regressor', Ridge())]),
    'Lasso Regression': Pipeline([('preprocessor', preprocessor), ('regressor', Lasso())]),
    'ElasticNet': Pipeline([('preprocessor', preprocessor), ('regressor', ElasticNet())]),
    'Random Forest Regression': Pipeline([('preprocessor', preprocessor), ('regressor', RandomForestRegressor())]),
    'Gradient Boosting Regression': Pipeline([('preprocessor', preprocessor), ('regressor', GradientBoostingRegressor())]),
    'SVR': Pipeline([('preprocessor', preprocessor), ('regressor', SVR())]),
}

# Paramètres pour GridSearchCV pour chaque modèle
parameters = {
    'Linear Regression': {'regressor__fit_intercept': [True,False]},
    'Ridge Regression': {'regressor__alpha': [0.1, 1.0, 10.0]},
    'Lasso Regression': {'regressor__alpha': [0.1, 1.0, 10.0]},
    'ElasticNet': {'regressor__alpha': [0.1, 1.0, 10.0], 'regressor__l1_ratio': [0.1, 0.5, 0.9]},
    'Random Forest Regression': {'regressor__n_estimators': [50, 100], 'regressor__max_depth': [10, 20]},# None, 
    'Gradient Boosting Regression': {'regressor__n_estimators': [50, 100, 200], 'regressor__max_depth': [3, 5, 7]},
    'SVR': {'regressor__kernel': ['linear', 'rbf'], 'regressor__C': [0.1, 1.0, 10.0]},
}

# Scoring : RMSE, R2 et MAE
scoring = {'RMSE': 'neg_root_mean_squared_error',
           'R2': 'r2',
           'MAE': 'neg_mean_absolute_error'}


In [20]:
def Grid(X_train, y_train, pipeline, parameters, cv=5):
  # Scoring
  #multi_scoring = {mean_squared_error,r2_score}
    
    
  # Grid search
  grid = GridSearchCV(pipeline, parameters,  scoring=scoring, refit='RMSE', cv=cv, n_jobs =-1, verbose = 1, error_score='raise')

  # Fit
  grid.fit(X_train, y_train)

  # Scores and results
  best_score = grid.best_score_.round(4)
  best_params = grid.best_params_
  training_time = grid.cv_results_['mean_fit_time'].mean().round(4)

  # Output
  return({
      'best_score': best_score,
      'best_params': best_params,
      'training_time': training_time,
      'fitted_model': grid.best_estimator_
  })

In [21]:
def afficheResults (grid):
    model_name = grid['fitted_model'].named_steps['regressor'].__class__.__name__
    print(f"{model_name} training time: {grid['training_time']}")
    print(f"Best {model_name} parameters: {grid['best_params']}")
    print(f"Best {model_name} score: {-grid['best_score']}")
 
    
    

In [22]:

# Boucle sur les modèles pour ajuster avec GridSearchCV
models = {}

# Boucle sur les modèles pour ajuster avec GridSearchCV et évaluation
for model_name, pipeline in pipelines.items():
    print(f"\n..............{model_name}..............................")
    grid_search = Grid(X_train, y_train, pipeline, parameters[model_name], cv=5)
    afficheResults (grid_search)
    
    best_model = grid_search['fitted_model']
    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    Rmse = round(math.sqrt(mse), 4)
    models[model_name] = [best_model,Rmse]
    print(f"{model_name} RMSE: {Rmse}")
 


..............Linear Regression..............................
Fitting 5 folds for each of 2 candidates, totalling 10 fits
LinearRegression training time: 4.1168
Best LinearRegression parameters: {'regressor__fit_intercept': True}
Best LinearRegression score: 2.386
Linear Regression RMSE: 1.4365

..............Ridge Regression..............................
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Ridge training time: 1.6303
Best Ridge parameters: {'regressor__alpha': 1.0}
Best Ridge score: 1.3125
Ridge Regression RMSE: 1.3189

..............Lasso Regression..............................
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Lasso training time: 3.0896
Best Lasso parameters: {'regressor__alpha': 0.1}
Best Lasso score: 1.4174
Lasso Regression RMSE: 1.4242

..............ElasticNet..............................
Fitting 5 folds for each of 9 candidates, totalling 45 fits
ElasticNet training time: 3.6186
Best ElasticNet parameters: {'regressor__alpha

In [ ]:
'''
# Boucle sur les modèles pour ajuster avec GridSearchCV
results = {}

# Boucle sur les modèles pour ajuster avec GridSearchCV et évaluation
for model_name, pipeline in pipelines.items():
    print(f"Training {model_name}...")
    grid_search = GridSearchCV(pipeline, parameters[model_name],  scoring=scoring, refit='RMSE', cv=5, n_jobs =-1, verbose = 1)
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)
    mse = root_mean_squared_error(y_test, y_pred)
    results[model_name] = mse
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Best {model_name} score: {-grid_search.best_score_}")
    print(f"{model_name} MSE: {mse}")
    print()
'''

'\n# Boucle sur les modèles pour ajuster avec GridSearchCV\nresults = {}\n\n# Boucle sur les modèles pour ajuster avec GridSearchCV et évaluation\nfor model_name, pipeline in pipelines.items():\n    print(f"Training {model_name}...")\n    grid_search = GridSearchCV(pipeline, parameters[model_name],  scoring=scoring, refit=\'RMSE\', cv=5, n_jobs =-1, verbose = 1)\n    grid_search.fit(X_train, y_train)\n    best_model = grid_search.best_estimator_\n    y_pred = best_model.predict(X_test)\n    mse = root_mean_squared_error(y_test, y_pred)\n    results[model_name] = mse\n    print(f"Best parameters: {grid_search.best_params_}")\n    print(f"Best {model_name} score: {-grid_search.best_score_}")\n    print(f"{model_name} MSE: {mse}")\n    print()\n'

In [ ]:
'''
# Evaluation des modèles sur les données de test
print("\nComparaison des performances des modèles sur les données de test:")
for model_name, mse in results.items():
    print(f"{model_name}: MSE = {mse}")
    
'''

'\n# Evaluation des modèles sur les données de test\nprint("\nComparaison des performances des modèles sur les données de test:")\nfor model_name, mse in results.items():\n    print(f"{model_name}: MSE = {mse}")\n    \n'